In [1]:
import os

import numpy as np
import pandas as pd

import torch
import torchvision

import matplotlib.pyplot as plt
import lightning as L

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from data_module.data_module import FFTDataModule
from model.enc_classifier_model import AE1DClassifier_simple

c:\Users\1001002996\AppData\Local\miniconda3\Lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
TEST = False

random_seed = 42
fft_dir_path = "dataset/processed_data"
AE1D_checkpoint_path = "lightning_logs/AE1D/version_2/checkpoints/sample_epoch=999-step=20000-val_mse=0.009731.ckpt"

L.seed_everything(random_seed)

Seed set to 42


42

In [3]:
if TEST:
    data_module = FFTDataModule(dataset_path=fft_dir_path)
    data_module.setup()

    print(len(data_module.train_set))
    print(len(data_module.val_set))
    print(len(data_module.test_set))

    val_loader = data_module.val_dataloader()
    print(next(iter(val_loader)))

In [5]:
if TEST:
    from lightning.pytorch.utilities.model_summary import ModelSummary

    net = AE1DClassifier_simple(
        AE1D_ckpt_path=AE1D_checkpoint_path,
        optimizer=optim.SGD,
        optimizer_param={
            "learning_rate": 0.01,
            "momentum": 0.5,
        }).to("cpu")
    model_summary = ModelSummary(net, max_depth=6)

    print(model_summary)

   | Name                           | Type            | Params | In sizes     | Out sizes   
--------------------------------------------------------------------------------------------------
0  | feature_extractor              | AE1D            | 4.0 M  | ?            | ?           
1  | feature_extractor.encoder      | Sequential      | 18.0 K | [10, 6, 257] | [10, 64, 26]
2  | feature_extractor.encoder.0    | Conv1d          | 1.6 K  | [10, 6, 257] | [10, 32, 84]
3  | feature_extractor.encoder.1    | ReLU            | 0      | [10, 32, 84] | [10, 32, 84]
4  | feature_extractor.encoder.2    | Dropout         | 0      | [10, 32, 84] | [10, 32, 84]
5  | feature_extractor.encoder.3    | Conv1d          | 16.4 K | [10, 32, 84] | [10, 64, 26]
6  | feature_extractor.encoder.4    | ReLU            | 0      | [10, 64, 26] | [10, 64, 26]
7  | feature_extractor.encoder.5    | Dropout         | 0      | [10, 64, 26] | [10, 64, 26]
8  | feature_extractor.enc_linear   | Sequential      | 2.0 M  |

In [17]:
n_epochs = 5000
patience = 50

In [18]:
print(" ----------------------start training---------------------------")
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger

tensorboard_logger = TensorBoardLogger(
    save_dir="lightning_logs",
    name="AE1DClassifier",
)

checkpoint_callback = ModelCheckpoint(
    dirpath=None,
    save_top_k=1,
    monitor="val_loss",
    mode="min",
    filename="sample_{epoch:02d}-{step:02d}-{val_loss:02f}"
)

trainer = L.Trainer(
    logger=tensorboard_logger,
    callbacks=[EarlyStopping(monitor="val_loss", patience=patience), checkpoint_callback],
    max_epochs=n_epochs,
    check_val_every_n_epoch=10,
    )

net = AE1DClassifier_simple(AE1D_ckpt_path=AE1D_checkpoint_path)
data_module = FFTDataModule(dataset_path=fft_dir_path, batch_size=8192)

trainer.fit(model=net, datamodule=data_module)
trainer.test(model=net, datamodule=data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 20 batch(es). Logging and checkpointing is suppressed.


 ----------------------start training---------------------------


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type       | Params | In sizes     | Out sizes   
-------------------------------------------------------------------------------
0 | feature_extractor | AE1D       | 4.0 M  | ?            | ?           
1 | encoder           | Sequential | 18.0 K | [10, 6, 257] | [10, 64, 26]
2 | enc_linear        | Sequential | 2.0 M  | [10, 1664]   | [10, 256]   
3 | cont_classifier   | Sequential | 33.9 K | [10, 256]    | [10, 8]     
-------------------------------------------------------------------------------
33.9 K    Trainable params
4.0 M     Non-trainable params
4.0 M     Total params
16.024    Total estimated model params size (MB)


Epoch 0:   0%|          | 0/20 [00:00<?, ?it/s] output.shape torch.Size([8192, 8])
output tensor([[-0.0755, -0.0065,  0.0758,  ..., -0.0193,  0.0359, -0.0395],
        [-0.0755, -0.0065,  0.0758,  ..., -0.0193,  0.0359, -0.0395],
        [-0.0730, -0.0225,  0.0727,  ..., -0.0056,  0.0530, -0.0512],
        ...,
        [-0.0755, -0.0065,  0.0758,  ..., -0.0193,  0.0359, -0.0395],
        [-0.0755, -0.0065,  0.0758,  ..., -0.0193,  0.0359, -0.0395],
        [-0.0755, -0.0065,  0.0758,  ..., -0.0193,  0.0359, -0.0395]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
y tensor([6, 3, 6,  ..., 7, 3, 2], device='cuda:0')
Epoch 0:   5%|▌         | 1/20 [00:00<00:05,  3.39it/s, train_loss=2.090]output.shape torch.Size([8192, 8])
output tensor([[-0.0655,  0.0035,  0.0658,  ..., -0.0093,  0.0459, -0.0495],
        [-0.0655,  0.0035,  0.0658,  ..., -0.0093,  0.0459, -0.0495],
        [-0.0144,  0.0332, -0.0259,  ...,  0.0483,  0.1100, -0.0561],
        ...,
        [-0.0655,  0.0035,  0.0658, 

`Trainer.fit` stopped: `max_steps=20` reached.


Epoch 0: 100%|██████████| 20/20 [00:02<00:00,  9.71it/s, train_loss=2.050, val_loss=2.040, val_acc=0.160]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 38.19it/s] 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.15959079563617706
        test_loss            2.038180351257324
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 2.038180351257324, 'test_acc': 0.15959079563617706}]